In [3]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

In [4]:
dict_rxn_to_gene["METLEUex"]

['6520.1', '8140.1']

In [5]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Non_masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)


edges_node_names_explainer_subgraphs

,node1,node2,0,1,2,3,4,5,6,7,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,r1643,asn_L_c,0.015867,0.016685,0.026855,0.027739,0.011675,0.027851,0.027855,0.265077,...,0.320273,0.026499,0.027669,0.113198,0.027240,0.026680,0.027890,0.013508,0.027602,0.027859
1,r1643,leu_L_c,0.015598,0.032502,0.025722,0.026414,0.014865,0.026811,0.026462,0.099351,...,0.314144,0.025764,0.026983,0.166961,0.025823,0.025755,0.027333,0.013611,0.026437,0.026763
2,r1500,HC02193_c,0.378324,0.157980,0.013878,0.013713,0.846867,0.013739,0.013797,0.565250,...,0.213469,0.013837,0.013717,0.226846,0.013779,0.013727,0.013673,0.502068,0.013683,0.013863
3,r1586,ser_L_c,0.019166,0.022074,0.025502,0.026028,0.013198,0.025835,0.025798,0.243078,...,0.274693,0.024883,0.025963,0.327898,0.025496,0.025358,0.026255,0.015075,0.025551,0.025854
4,r1654,asn_L_c,0.018128,0.017420,0.027166,0.027353,0.011751,0.027045,0.026923,0.325512,...,0.351430,0.026267,0.026558,0.122128,0.026799,0.026554,0.027376,0.014920,0.027257,0.026937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,KYN,Lkynr_c,0.008546,0.077920,0.013805,0.013743,0.008893,0.013741,0.013859,0.124245,...,0.107213,0.013787,0.013761,0.088647,0.013797,0.013589,0.013667,0.006961,0.013801,0.013829
591,SERDGLYexR,ser_D_e,0.012614,0.012650,0.013851,0.013831,0.012128,0.013850,0.013845,0.896968,...,0.879889,0.013756,0.013750,0.919603,0.013879,0.013781,0.013834,0.012488,0.013695,0.013748
592,DALAxt,ala_D_c,0.078425,0.098075,0.013880,0.013774,0.154436,0.013901,0.013742,0.161404,...,0.165409,0.013785,0.013817,0.151732,0.013843,0.013711,0.013727,0.067422,0.013851,0.013866
593,r1813,phe_L_e,0.009363,0.024050,0.478146,0.525948,0.007765,0.471708,0.492006,0.046023,...,0.040722,0.489982,0.470612,0.035508,0.505931,0.480945,0.499485,0.010140,0.472664,0.496916


In [6]:

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]


2022-10-27 08:51:36,158	INFO worker.py:1518 -- Started a local Ray instance.


In [7]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R.query("node2 == 'leuktrF4_c'")

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
29,[387775.1],LEUKTRF4t,leuktrF4_c,0.907831,0.932333,0.013798,0.013863,0.212747,0.013620,0.013945,...,0.750076,0.013818,0.013711,0.658600,0.013904,0.013859,0.013663,0.820294,0.013756,0.013808
84,[],GGT6,leuktrF4_c,0.976260,0.938191,0.013780,0.013799,0.982821,0.013864,0.013810,...,0.819138,0.013707,0.013657,0.027888,0.013901,0.013706,0.013771,0.984148,0.013636,0.013839
171,[],LTC4CP,leuktrF4_c,0.983192,0.744792,0.013833,0.013592,0.983500,0.013770,0.013873,...,0.881690,0.013938,0.013732,0.927749,0.013850,0.013847,0.013776,0.985584,0.013840,0.013648
321,[10599.1],r2251,leuktrF4_c,0.113978,0.123500,0.013736,0.013816,0.237809,0.013842,0.013736,...,0.189160,0.013777,0.013771,0.198581,0.013719,0.013898,0.013826,0.123732,0.013812,0.013704
576,[10599.1],r2252,leuktrF4_c,0.112991,0.123346,0.013834,0.013693,0.239645,0.013708,0.013911,...,0.189962,0.013711,0.013813,0.199445,0.013676,0.013763,0.013750,0.123722,0.013723,0.013694
587,[6578.1],r2318,leuktrF4_c,0.049569,0.021781,0.013895,0.013646,0.326519,0.013789,0.013714,...,0.046297,0.013912,0.013773,0.076283,0.013728,0.013710,0.013974,0.063046,0.013619,0.013927
594,[10599.1],r2253,leuktrF4_c,0.113239,0.124337,0.013675,0.013810,0.243089,0.013705,0.013757,...,0.191700,0.013769,0.013768,0.197128,0.013754,0.013790,0.013730,0.124064,0.013695,0.013712


In [8]:
results_for_R.to_csv("./results/data/data_for_R/Non_masked_results_for_R.csv")

# Masked

In [13]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo
    
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)
ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]



2022-10-27 08:56:31,250	INFO worker.py:1518 -- Started a local Ray instance.


In [14]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R.query("node1 == 'METLEUex'")

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
116,"[6520.1, 8140.1]",METLEUex,leu_L_c,0.077435,0.086118,0.012916,0.013739,0.013346,0.494376,0.013851,...,0.013385,0.013734,0.013786,0.013692,0.157411,0.321533,0.013837,0.013208,0.175792,0.013687
117,"[6520.1, 8140.1]",METLEUex,leu_L_e,0.193003,0.277794,0.012819,0.012130,0.012971,0.264593,0.013526,...,0.012563,0.013135,0.012161,0.012184,0.260260,0.271496,0.013974,0.013417,0.255833,0.013323


In [15]:
results_for_R.to_csv("./results/data/data_for_R/Masked_results_for_R.csv")